In [ ]:
import pandas as pd
data = pd.read_csv("C:\\Users\\NB332021\\OneDrive - Nedbank\\Desktop\\Thesis\\Data\\ConsFinal.csv")
 

In [ ]:
#checking the data formatting
data["created_at"]

In [ ]:
#checking the shape of data
data.shape

In [ ]:
# drop the first row
data.drop(index=0, inplace=True)

#### Now convert the data values

In [ ]:
def handle_non_date_values(df, column_name):
    for i in df.index:
        try:
            df.loc[i, column_name] = pd.to_datetime(df.loc[i, column_name])
        except ValueError:
            if i > 0:
                df.loc[i, column_name] = df.loc[i-1, column_name]
            else:
                df.loc[i, column_name] = df.loc[i+1, column_name]
    return df
 

In [ ]:
handle_non_date_values(data, "created_at")

In [ ]:
from datetime import datetime
 
# assume 'df' is your DataFrame and 'column_name' is the column you want to format
data['created_at'] = pd.to_datetime(data['created_at']).dt.strftime('%b-%Y')

In [ ]:
data.to_csv("C:\\Users\\Desktop\\Thesis\\Data\\datesCorrect")

##### Sentiment package: VaderSentiment

##### Prep data for sentiment analysis

In [ ]:
#perform imports and load dataset
import numpy as np
import pandas as pd
import scipy
import spacy
import re # For regular expressions|||
import string # For handling string
import nltk
import vaderSentiment
#import matplotlib.pyplot as plt
from nltk.util import ngrams
#import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter
 
#data preprocessing:
df1 = data.copy()
#CLEANING DATA
#drop duplicates
data = data.drop_duplicates(keep='first')
#removing numbers
df1['text'] = df1['text'].astype('string')
df1.dropna(subset=['text'], inplace=True)
#lowercase
df1['text']=df1['text'].apply(lambda x: x.lower())
#removing mentions
df1['text']=df1['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+",'', x))
#removing numbers and words containing numbers
df1['text']=df1['text'].apply(lambda x: re.sub('\w*\d\w*','', x))
#removing hashtags?
#df1['text']=df1['text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+",'', x))
#df1['text']=df1['text'].apply(lambda x: re.split('#|_', x))
df1['text']=df1['text'].apply(lambda x: " ".join(word.strip() for word in re.split('#|_', x)))
#removing links
df1['text']=df1['text'].apply(lambda x: re.sub(r"http\S+",'', x))
df1['text']=df1['text'].apply(lambda x: re.sub(r"www.\S+",'', x))
#removing punctuation
df1['text']=df1['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#removing empty spaces and special characters
df1['text']=df1['text'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))
#stopwords
nlp = spacy.load('en_core_web_md')
# Print the set of spaCy's default stop words (remember that sets are unordered):
print(nlp.Defaults.stop_words)
#add twitter as a stop word and rt
# Add the word to the set of stop words. Use LOWERCASE!
nlp.Defaults.stop_words.add('twitter')
nlp.Defaults.stop_words.add('rt')
nlp.Defaults.stop_words.add('s')
nlp.Defaults.stop_words.add('tweet')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('follower')
nlp.Defaults.stop_words.add('null')
nlp.Defaults.stop_words.add('http')
nlp.Defaults.stop_words.add('url')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('trending')
# Set the stop_word tag on the lexeme
nlp.vocab['twitter'].is_stop = True
nlp.vocab['rt'].is_stop = True
nlp.vocab['s'].is_stop = True
nlp.vocab['tweet'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['follower'].is_stop = True
nlp.vocab['null'].is_stop = True
nlp.vocab['http'].is_stop = True
nlp.vocab['url'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['trending'].is_stop = True
stop = nlp.Defaults.stop_words
#lemmatisation
# Loading model
#nlp = spacy.load('en_core_web_sm')
 
# Lemmatization with stopwords removal
#df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False)&((token.pos_=="ADJ")|(token.pos_=="NOUN")|(token.pos_=="VERB")))]))
#df1.head()
 
df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False))]))
 
#getting rid of empty text rows
df1 = df1.dropna(subset=['text'])
 
data = df1
 
del df1

#### VaderSentiment sentiment

In [ ]:
#libraries
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
#VADER SENTIMENT
#sentiment column
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.

In [ ]:
sid_obj = SentimentIntensityAnalyzer()
#pip install vaderSentiment

In [ ]:
#add a new column
lists = list(range(len(data)))
data["Sentiment"] = lists

In [ ]:
#aply function to get sentiment for each row
#df1['Sentiment'] = df1['text'].apply(lambda x: sid_obj.polarity_scores(x)['compound'])
result = []

In [ ]:
data["text"] = pd.DataFrame(data["text"] )
type(data["text"] )


In [ ]:
for index, row in data.iterrows():
     text = row['text']
     sentiment  = sid_obj.polarity_scores(text)['compound']
     result.append((sentiment))
 
data['VaderSentiment'] = result
data.head()

In [ ]:
print(data.loc[1,"created_at"])

#### Create VaderSentiment dataset to be used in time series comparison

In [ ]:
datacopy = data.copy()
datacopy['month'] = datacopy['created_at'].dt.to_period('M')
monthly_sentiment = datacopy.groupby('month')['VaderSentiment'].mean()
monthly_sentiment = pd.DataFrame(monthly_sentiment)
monthly_sentiment.to_csv("C:\\Users\\NB332021\\OneDrive - Nedbank\\Desktop\\Thesis\\Data\\monthlyvsent.csv")

In [ ]:
#plot the monthly sentiment
import matplotlib.pyplot as plt
monthly_sentiment.plot(kind='line', figsize=(10,6), marker='o', linestyle='-', color='blue')
plt.title('Monthly Sentiment')
plt.xlabel('Month')
plt.ylabel('Average Sentiment')
plt.grid(True)
plt.show()

#Vaderssentiment is between -1 and 1